In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from urllib.parse import urljoin

# ========== CONFIG ==========
car_model = "پژو 504"
city = "tehran"
scroll_count = 20
scroll_pause = 2

# ========== SETUP ==========
service = Service(r"C:\Users\Asus\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")  # Make sure chromedriver is in your folder
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(f"https://divar.ir/s/{city}/auto?q={car_model}")
time.sleep(3)

# ========== DATA COLLECTION ==========
seen_links = set()
results = []

for i in range(scroll_count):
    print(f"🔄 Scroll {i + 1}/{scroll_count}")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)

    # Try clicking "آگهی‌های بیشتر"
    try:
        show_more = driver.find_element(By.CLASS_NAME, "post-list__load-more-btn-be092")
        if show_more.is_displayed():
            print("🟢 Clicking 'آگهی‌های بیشتر' button")
            driver.execute_script("arguments[0].click();", show_more)
            time.sleep(scroll_pause)
    except NoSuchElementException:
        print("🔕 No 'Show More' button found")

    # Extract title and URL from visible posts
    cards = driver.find_elements(By.CLASS_NAME, "widget-col-d2306")
    print(f"📦 Found {len(cards)} visible cards")

    for card in cards:
        try:
            link_tag = card.find_element(By.TAG_NAME, "a")
            link = link_tag.get_attribute("href")

            if not link or link in seen_links:
                continue
            seen_links.add(link)

            title_tag = card.find_element(By.CLASS_NAME, "kt-post-card__title")
            title = title_tag.text.strip()

            results.append((title, link))
        except Exception as e:
            continue

    print(f"📈 Total unique posts collected so far: {len(results)}\n")

# ========== FILTER VALID CARS ==========
valid_keywords = [
"504","۵۰۴"
]

car_posts = [(title, url) for title, url in results if any(k in title for k in valid_keywords)]

print(f"\n✅ Final Valid Car Posts: {len(car_posts)} out of {len(results)}")
print("=============================================")
for title, url in car_posts:
    print(f"🟢 {title}")
    print(f"🔗 {url}\n")

driver.quit()


🔄 Scroll 1/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 2/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 3/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 4/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 5/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 6/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 7/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 8/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so far: 19

🔄 Scroll 9/20
🔕 No 'Show More' button found
📦 Found 22 visible cards
📈 Total unique posts collected so f

In [ ]:

# your code here...

# when all done
# driver.quit()


In [4]:
import os
import requests
from PIL import Image
from io import BytesIO
import time
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(r"C:\Users\Asus\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://divar.ir")  # test open

# Your car model keyword to check in the post
target_car_model = ""# change this to your desired model keyword
not_in_model="SD"
max_files = 640
save_dir = "images"
os.makedirs(save_dir, exist_ok=True)

for idx, (car_title, car_url) in enumerate(car_posts, 1):
    print(f"\n🚗 Processing car #{idx}: {car_title}\n🔗 {car_url}")

    try:
        driver.get(car_url)
        time.sleep(3)  # wait for page to load

        # Double check the car model in specific element's text
        try:
            model_check_elem = driver.find_element(By.CLASS_NAME, "kt-unexpandable-row__action.kt-text-truncate")
            model_text = model_check_elem.text
            if target_car_model not in model_text:
                print(f"⏭️ Skipped post because model check failed: found '{model_text}'")
                continue
        except Exception as e:
            print(f"⚠️ Couldn't find model check element: {e}")
            continue  # skip if can't find element

        image_elements = driver.find_elements(By.CLASS_NAME, "kt-image-block__image--fading")
        print(f"🖼️ Found {len(image_elements)} images.")

        for i, img in enumerate(image_elements):
            file_count = len([f for f in os.listdir(save_dir) if os.path.isfile(os.path.join(save_dir, f))])
            if file_count >= max_files:
                print(f"🚫 Directory '{save_dir}' already has {file_count} files. Stopping script.")
                driver.quit()
                sys.exit()  

            try:
                img_url = img.get_attribute("src")
                if not img_url or not img_url.startswith("http"):
                    continue

                response = requests.get(img_url)
                image = Image.open(BytesIO(response.content))
                ext = image.format.lower()

                if ext not in ["jpeg", "jpg", "png"]:
                    print(f"⏭️ Skipping unsupported image format: {ext}")
                    continue

                filename = os.path.join(save_dir, f"car_{idx}_img{i + 1}.{ext}")
                image.save(filename)
                print(f"✅ Saved: {filename}")

            except Exception as e:
                print(f"❌ Error saving image #{i + 1} for post {idx}: {e}")

    except Exception as e:
        print(f"❌ Failed to process post {idx}: {e}")



🚗 Processing car #1: پژو 504 بهترین رنگ
🔗 https://divar.ir/v/%D9%BE%DA%98%D9%88-504-%D8%A8%D9%87%D8%AA%D8%B1%DB%8C%D9%86-%D8%B1%D9%86%DA%AF/Aa7ZtAWb
⚠️ Couldn't find model check element: Message: no such element: Unable to locate element: {"method":"css selector","selector":".kt-unexpandable-row__action.kt-text-truncate"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff768e56f75+76917]
	GetHandleVerifier [0x0x7ff768e56fd0+77008]
	(No symbol) [0x0x7ff768c09dea]
	(No symbol) [0x0x7ff768c60256]
	(No symbol) [0x0x7ff768c6050c]
	(No symbol) [0x0x7ff768cb3887]
	(No symbol) [0x0x7ff768c884af]
	(No symbol) [0x0x7ff768cb065c]
	(No symbol) [0x0x7ff768c88243]
	(No symbol) [0x0x7ff768c51431]
	(No symbol) [0x0x7ff768c521c3]
	GetHandleVerifier [0x0x7ff76912d2ad+3051437]
	GetHandleVerifier [0x0x7ff769127903+3028483]
	GetHandleVe

In [1]:
driver.quit()


NameError: name 'driver' is not defined

In [5]:
import os
import requests
from PIL import Image
from io import BytesIO
import time
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

service = Service(r"C:\Users\Asus\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://divar.ir")  # test open

# Your car model keyword to check in the post
target_car_model = ""# change this to your desired model keyword
not_in_model="SD"
max_files = 640
save_dir = "images"
os.makedirs(save_dir, exist_ok=True)

for idx, (car_title, car_url) in enumerate(car_posts, 1):
    print(f"\n🚗 Processing car #{idx}: {car_title}\n🔗 {car_url}")

    try:
        driver.get(car_url)
        time.sleep(3)  # wait for page to load

        # Double check the car model in specific element's text
        try:
            model_check_elem = driver.find_element(By.CLASS_NAME, "kt-unexpandable-row__action.kt-text-truncate")
            model_text = model_check_elem.text

        except Exception as e:
            print(f"⚠️ Couldn't find model check element: {e}")
            continue  # skip if can't find element

        image_elements = driver.find_elements(By.CLASS_NAME, "kt-image-block__image--fading")
        print(f"🖼️ Found {len(image_elements)} images.")

        for i, img in enumerate(image_elements):
            file_count = len([f for f in os.listdir(save_dir) if os.path.isfile(os.path.join(save_dir, f))])
            if file_count >= max_files:
                print(f"🚫 Directory '{save_dir}' already has {file_count} files. Stopping script.")
                driver.quit()
                sys.exit()  

            try:
                img_url = img.get_attribute("src")
                if not img_url or not img_url.startswith("http"):
                    continue

                response = requests.get(img_url)
                image = Image.open(BytesIO(response.content))
                ext = image.format.lower()

                if ext not in ["jpeg", "jpg", "png"]:
                    print(f"⏭️ Skipping unsupported image format: {ext}")
                    continue

                filename = os.path.join(save_dir, f"car_{idx}_img{i + 1}.{ext}")
                image.save(filename)
                print(f"✅ Saved: {filename}")

            except Exception as e:
                print(f"❌ Error saving image #{i + 1} for post {idx}: {e}")

    except Exception as e:
        print(f"❌ Failed to process post {idx}: {e}")



🚗 Processing car #1: پژو 504 بهترین رنگ
🔗 https://divar.ir/v/%D9%BE%DA%98%D9%88-504-%D8%A8%D9%87%D8%AA%D8%B1%DB%8C%D9%86-%D8%B1%D9%86%DA%AF/Aa7ZtAWb
⚠️ Couldn't find model check element: Message: no such element: Unable to locate element: {"method":"css selector","selector":".kt-unexpandable-row__action.kt-text-truncate"}
  (Session info: chrome=138.0.7204.158); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x7ff768e56f75+76917]
	GetHandleVerifier [0x0x7ff768e56fd0+77008]
	(No symbol) [0x0x7ff768c09dea]
	(No symbol) [0x0x7ff768c60256]
	(No symbol) [0x0x7ff768c6050c]
	(No symbol) [0x0x7ff768cb3887]
	(No symbol) [0x0x7ff768c884af]
	(No symbol) [0x0x7ff768cb065c]
	(No symbol) [0x0x7ff768c88243]
	(No symbol) [0x0x7ff768c51431]
	(No symbol) [0x0x7ff768c521c3]
	GetHandleVerifier [0x0x7ff76912d2ad+3051437]
	GetHandleVerifier [0x0x7ff769127903+3028483]
	GetHandleVe

In [7]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

query = "Peugeot 504 2000"
save_dir = "504_2000_images"
scroll_count = 10  # increase for more

# Setup
service = Service(r"C:\Users\Asus\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe")
os.makedirs(save_dir, exist_ok=True)
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

# Search on Google
search_url = f"https://www.google.com/search?q={query}&tbm=isch"
driver.get(search_url)
time.sleep(2)

# Scroll
for _ in range(scroll_count):
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    time.sleep(1)

# Get images
images = driver.find_elements(By.CSS_SELECTOR, "img.rg_i")
print(f"Found {len(images)} images.")

count = 0
for img in images:
    try:
        src = img.get_attribute("src") or img.get_attribute("data-src")
        if src and "http" in src:
            img_data = requests.get(src).content
            with open(os.path.join(save_dir, f"{count}.jpg"), "wb") as f:
                f.write(img_data)
            count += 1
        if count >= 200:
            break
    except:
        pass

driver.quit()
print(f"Downloaded {count} images.")


ReadTimeoutError: HTTPConnectionPool(host='localhost', port=55689): Read timed out. (read timeout=120)